# Lux AI Season 1 Python Tutorial Notebook

Welcome to Lux AI Season 1!

This notebook is the basic setup to use Jupyter Notebooks and the `kaggle-environments` package to develop your bot. If you plan to not use Jupyter Notebooks or any other programming language, please see our [Github](https://github.com/Lux-AI-Challenge/Lux-Design-2021). The following are some important links!

- Competition Page: https://www.kaggle.com/c/lux-ai-2021/

- Online Visualizer: https://2021vis.lux-ai.org/

- Specifications: https://www.lux-ai.org/specs-2021

- Github: https://github.com/Lux-AI-Challenge/Lux-Design-2021

- Bot API: https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits

And if you haven't done so already, we **highly recommend** you join our Discord server at https://discord.gg/aWJt3UAcgn or at the minimum follow the kaggle forums at https://www.kaggle.com/c/lux-ai-2021/discussion. We post important announcements there such as changes to rules, events, and opportunities from our sponsors!

Now let's get started!

## Prerequisites

We assume that you have a basic knowledge of Python and programming. It's okay if you don't know the game specifications yet! Feel free to always refer back to https://www.lux-ai.org/specs-2021.

## Basic Setup

First thing to verify is that you have **Node.js v12 or above**. The engine for the competition runs on Node.js (for many good reasons including an awesome visualizer) and thus it is required. You can download it [here](https://nodejs.org/en/download/). You can then verify you have the appropriate version by running


In [ ]:
!node -v

We will also need Kaggle Environments

In [ ]:
!pip install kaggle-environments -U

Next, we have to import the `make` function from the `kaggle_environments` package

In [ ]:
from kaggle_environments import make

## Building from Scratch

The following bit of code is all you need for a empty agent that does nothing

In [ ]:
# run this if using kaggle notebooks
!cp -r ../input/lux-ai-2021/* .
# if working locally, download the `simple/lux` folder from here https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits/python
# and we recommend following instructions in there for local development with python bots

In [ ]:
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

# we declare this global game_state object so that state persists across turns so we do not need to reinitialize it all the time
game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    # add debug statements like so!
    if game_state.turn == 0:
        print("Agent is running!", file=sys.stderr)
        actions.append(annotate.circle(0, 0))
    return actions

In [ ]:
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state,player):
    resource_tiles: list[Position] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                if cell.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
                if cell.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
                resource_tiles.append(cell.pos)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        dist = resource_tile.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    
    if closest_resource_tile in resource_tiles:
        resource_tiles.remove(closest_resource_tile)       
    
    return closest_resource_tile, resource_tiles

In [ ]:
# snippet to find the closest city tile to a position
def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
                    
    return closest_city_tile

In [ ]:
# Поиск ближайшей ячейки города, которому не хватает ресурсов
def find_closest_city_not_ready_for_night_tile(pos, player, cities_not_ready_for_night):
    closest_city_tile = None
    cid = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            if k not in cities_not_ready_for_night: continue
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
                    cid = k
        if cid in cities_not_ready_for_night:
             cities_not_ready_for_night.remove(cid)
   
    return closest_city_tile, cities_not_ready_for_night

In [ ]:
def update_cities_not_ready_for_night(player, cities_not_ready_for_night):
    in_city_unit_id = []
    last_city_id = None
    if len(player.cities) > 0 and cities_not_ready_for_night is not None:       
        for k, city in player.cities.items():
            if k not in cities_not_ready_for_night: continue
            for city_tile in city.citytiles:
                for unit in player.units:
                    if unit.pos.distance_to(city_tile.pos) == 0 and is_resours_near(game_state, unit.pos) and (last_city_id is None or last_city_id != k):
                        in_city_unit_id.append(unit.id)
                        last_city_id = k
                        continue
            if last_city_id == k:
                cities_not_ready_for_night.append(k)
    return in_city_unit_id, cities_not_ready_for_night

In [ ]:
# Поиск всех городов, которым не хватает на ночь ресурсов
def find_cities_not_ready_for_night(player):
    city_not_ready_for_night = []
    if len(player.cities) > 0:
        for k, city in player.cities.items():
            if city.get_light_upkeep() * 10.0 > city.fuel:
                city_not_ready_for_night.append(k)
    return city_not_ready_for_night

In [ ]:
#Считаем количество ресурсов для композиции городов на 1 ночь
def city_resours_for_night(player):
    resourses = []
    print (player.cities)
    if len(player.cities) > 0:
        for k, city in player.cities.items():
            res_temp = 0
            for city_tile0 in city.citytiles:
                count_adjacent = 0
                for city_tile1 in city.citytiles:
                    dist = city_tile1.pos.distance_to(city_tile0.pos)
                    if dist == 1.0:
                        count_adjacent += 1 
                res_temp += 23 - 5 * count_adjacent
            resourses.append(res_temp)
    return resourses

In [ ]:
# Поиск клетки без ресурса с, как минимум, двумя смежными и одним доступным ресурсом
def find_adjacent_resources(game_state, player):
    adjacent_resource_tiles: list[Position] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height)[1:-1]:
        for x in range(width)[:-1]:
            cell = game_state.map.get_cell(x, y)
            cell0 = game_state.map.get_cell(x + 1, y)
            if cell.has_resource() and not cell0.has_resource():
                if cell.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
                if cell.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
                if game_state.map.get_cell(x + 1, y + 1).has_resource() or game_state.map.get_cell(x + 1, y - 1).has_resource():
                    adjacent_resource_tiles.append(cell0.pos)
    return adjacent_resource_tiles

In [ ]:
# Поиск пустых наиближайших пустых плиток с двумя смежными доступнымми ресурсами к центру
def find_closest_adjacent_resources_near_center(game_state, pos, adjacent_resources):
    closest_resource_tile = None
    closest_dist = math.inf
    cx, cy = game_state.map_width / 2, game_state.map_height / 2
    center: list[Position] = [Position(cx, cy), Position(cx - 1, cy - 1), Position(cx - 1, cy), Position(cx, cy - 1)]
    for c in center:
        dist = c.distance_to(pos)
        if dist < closest_dist:
                closest_dist = dist
                c0 = c
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in adjacent_resources:
        dist = resource_tile.distance_to(c0)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    if closest_resource_tile in adjacent_resources:
        adjacent_resources.remove(closest_resource_tile)       
    
    return closest_resource_tile, adjacent_resources

In [ ]:
def is_resours_near(game_state, pos):
    x = pos.x
    y = pos.y
    if x > 1 and game_state.map.get_cell(x - 1, y).has_resource(): return True
    if x < game_state.map_width and game_state.map.get_cell(x + 1, y).has_resource(): return True
    if y > 1 and game_state.map.get_cell(x, y - 1).has_resource(): return True
    if y < game_state.map_height and game_state.map.get_cell(x , y + 1).has_resource(): return True
    return False

In [ ]:
def next_pos_for_unit_who_need_move(unit_need_move, unit_pos_next, go_to):
    random_dir = []
    actions = []
    if len(unit_need_move) > 0:
        for unit in unit_need_move:
            random_dir = []
            x = unit.pos.x
            y = unit.pos.y

            if go_to.x > x and unit.pos.translate('e', 1) not in unit_pos_next:
                action = unit.move('e')
                actions.append(action)
                continue
            if go_to.x < x and unit.pos.translate('w', 1) not in unit_pos_next:
                action = unit.move('w')
                actions.append(action)
                continue
            if go_to.x > x and unit.pos.translate('s', 1) not in unit_pos_next:
                action = unit.move('s')
                actions.append(action)
                continue
            if go_to.x > x and unit.pos.translate('n', 1) not in unit_pos_next:
                action = unit.move('n')
                actions.append(action)
                continue

            if x > 1 and unit.pos.translate('w', 1) not in unit_pos_next: random_dir.append('w')
            if x < game_state.map_width and unit.pos.translate('e', 1) not in unit_pos_next : random_dir.append('e')
            if y > 1 and unit.pos.translate('n', 1) not in unit_pos_next : random_dir.append('n')
            if y < game_state.map_height and unit.pos.translate('s', 1) not in unit_pos_next : random_dir.append('s')

            while len(random_dir) > 0:
                rnd = random.choice(random_dir)
                if unit.pos.translate(rnd, 1) not in unit_pos_next: 
                    actions.append(unit.move(rnd))
                    break
                else:
                    random_dir.remove(rnd)
    return actions

In [ ]:
game_state = None
def agent1(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    # add debug statements like so!
    if game_state.turn == 0:
        print("Agent is running!", file=sys.stderr)

    occupied_cells: list[Position] = []
    cur_unit_pos: list[Position] = []
    unit_pos_next: list[Position] = []
    unit_need_move: list[player.units] = []
    all_citytiles_pos: list[Position] = []
    go_to: list[Position] = []
    unit_stand_in_city = []
    
    resource_tiles = find_resources(game_state,player)
    cities_not_ready_for_night = find_cities_not_ready_for_night(player)
    unit_stand_in_city, cities_not_ready_for_night =  update_cities_not_ready_for_night(player, cities_not_ready_for_night)
    adjacent_resources = find_adjacent_resources(game_state, player)
    
    citytiles_sum = 0
    for k, city in player.cities.items():
        for ct0 in city.citytiles:
            all_citytiles_pos.append(ct0.pos)
            citytiles_sum += 1

    for unit in player.units:
        cur_unit_pos.append(unit.pos)
        if unit.id in unit_stand_in_city: continue
        closest_city_tile = find_closest_city_tile(unit.pos, player)
        closest_city_not_ready_for_night_tile, cities_not_ready_for_night = find_closest_city_not_ready_for_night_tile(unit.pos, player, cities_not_ready_for_night)
        if closest_city_not_ready_for_night_tile is not None:
            if unit.pos.distance_to(closest_city_not_ready_for_night_tile.pos) < 10 and unit.get_cargo_space_left() > 25:
                if unit.is_worker() and unit.can_act(): 
                    next_pos = unit.pos.translate(unit.pos.direction_to(closest_city_not_ready_for_night_tile.pos), 1)
                    if next_pos not in unit_pos_next or next_pos in all_citytiles_pos:
                        action = unit.move(unit.pos.direction_to(closest_city_not_ready_for_night_tile.pos))
                        actions.append(action)
                        unit_pos_next.append(next_pos)
                        
                    else:
                        go_to.append(closest_city_not_ready_for_night_tile.pos)
                        unit_need_move.append(unit)   
                    
                    continue
 
        if unit.is_worker() and unit.can_act(): 
            if unit.get_cargo_space_left() > 0:
                closest_resource_tile, adjacent_resources = find_closest_adjacent_resources_near_center(game_state, unit.pos, adjacent_resources)
                closest_resource_tile_one, resource_tiles = find_closest_resources(unit.pos, resource_tiles)
                if closest_resource_tile is not None:
                    next_pos = unit.pos.translate(unit.pos.direction_to(closest_resource_tile), 1)
                    if next_pos not in unit_pos_next or next_pos in all_citytiles_pos:
                        action = unit.move(unit.pos.direction_to(closest_resource_tile))
                        actions.append(action)
                        unit_pos_next.append(next_pos)
                    else:
                        go_to.append(closest_resource_tile)
                        unit_need_move.append(unit)       
                else:
                    if closest_resource_tile_one is not None:
                        next_pos = unit.pos.translate(unit.pos.direction_to(closest_resource_tile_one), 1)
                        if next_pos not in unit_pos_next or next_pos in all_citytiles_pos:
                            action = unit.move(unit.pos.direction_to(closest_resource_tile_one))
                            actions.append(action)
                            unit_pos_next.append(next_pos)
                            
                        else:
                            unit_need_move.append(unit)  
                            go_to.append(closest_resource_tile_one)
            else:
                if unit.can_build(game_state.map) : 
                    d = unit.pos.distance_to(closest_city_tile.pos)
                    if d > 4 or d < 2:
                        action = unit.build_city()
                        actions.append(action)
                    else:
                        next_pos = unit.pos.translate(unit.pos.direction_to(closest_city_tile.pos), 1)
                        if next_pos not in unit_pos_next or next_pos in all_citytiles_pos:
                            action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                            actions.append(action)
                            unit_pos_next.append(next_pos)
                        else:
                            unit_need_move.append(unit)  
                            go_to.append(closest_city_tile.pos)
                        
                else:
                    if closest_city_tile is not None:
                        next_pos = unit.pos.translate(unit.pos.direction_to(closest_city_tile.pos), 1)
                        if next_pos not in unit_pos_next or next_pos in all_citytiles_pos:
                            action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                            actions.append(action)
                            unit_pos_next.append(next_pos)
                        else:
                            unit_need_move.append(unit)  
                            go_to.append(closest_city_tile.pos)
        else:
            unit_pos_next.append(unit.pos)
            
        if unit.pos in resource_tiles:
            resource_tiles.remove(unit.pos)
        if unit.pos in adjacent_resources:
            adjacent_resources.remove(unit.pos)
                                                      
    for k, city in player.cities.items():
        for ct0 in city.citytiles:
            if ct0.can_act():
                if ct0.pos not in cur_unit_pos and citytiles_sum > len(player.units):
                    action = ct0.build_worker()
                    actions.append(action)
                else:
                    action = ct0.research()
                    actions.append(action)
    
    return actions

In [ ]:
game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    # add debug statements like so!
    if game_state.turn == 0:
        print("Agent is running!", file=sys.stderr)

    occupied_cells: list[Position] = []
    cur_unit_pos: list[Position] = []
    unit_pos_next: list[Position] = []
    unit_need_move: list[player.units] = []
    all_citytiles_pos: list[Position] = []
    go_to: list[Position] = []
    unit_stand_in_city = []
    
    resource_tiles = find_resources(game_state,player)
    cities_not_ready_for_night = find_cities_not_ready_for_night(player)
    unit_stand_in_city, cities_not_ready_for_night =  update_cities_not_ready_for_night(player, cities_not_ready_for_night)
    adjacent_resources = find_adjacent_resources(game_state, player)
    
    citytiles_sum = 0
    for k, city in player.cities.items():
        for ct0 in city.citytiles:
            all_citytiles_pos.append(ct0.pos)
            citytiles_sum += 1

    for unit in player.units:
        cur_unit_pos.append(unit.pos)
        if unit.id in unit_stand_in_city: continue
        closest_city_tile = find_closest_city_tile(unit.pos, player)
        closest_city_not_ready_for_night_tile, cities_not_ready_for_night = find_closest_city_not_ready_for_night_tile(unit.pos, player, cities_not_ready_for_night)
        if closest_city_not_ready_for_night_tile is not None:
            if unit.pos.distance_to(closest_city_not_ready_for_night_tile.pos) < 10 and unit.get_cargo_space_left() > 25:
                if unit.is_worker() and unit.can_act(): 
                    next_pos = unit.pos.translate(unit.pos.direction_to(closest_city_not_ready_for_night_tile.pos), 1)
                    action = unit.move(unit.pos.direction_to(closest_city_not_ready_for_night_tile.pos))
                    actions.append(action)
                    unit_pos_next.append(next_pos)   
                    
                    continue
 
        if unit.is_worker() and unit.can_act(): 
            if unit.get_cargo_space_left() > 0:
                closest_resource_tile, adjacent_resources = find_closest_adjacent_resources_near_center(game_state, unit.pos, adjacent_resources)
                closest_resource_tile_one, resource_tiles = find_closest_resources(unit.pos, resource_tiles)
                if closest_resource_tile is not None:
                    next_pos = unit.pos.translate(unit.pos.direction_to(closest_resource_tile), 1)
                    action = unit.move(unit.pos.direction_to(closest_resource_tile))
                    actions.append(action)
                    unit_pos_next.append(next_pos)
    
                else:
                    if closest_resource_tile_one is not None:
                        next_pos = unit.pos.translate(unit.pos.direction_to(closest_resource_tile_one), 1)
                        action = unit.move(unit.pos.direction_to(closest_resource_tile_one))
                        actions.append(action)
                        unit_pos_next.append(next_pos)
            else:
                if unit.can_build(game_state.map) : 
                    d = unit.pos.distance_to(closest_city_tile.pos)
                    if d > 4 or d < 2:
                        action = unit.build_city()
                        actions.append(action)
                    else:
                        next_pos = unit.pos.translate(unit.pos.direction_to(closest_city_tile.pos), 1)
                        action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                        actions.append(action)
                        unit_pos_next.append(next_pos)
                            
                        
                else:
                    if closest_city_tile is not None:
                        next_pos = unit.pos.translate(unit.pos.direction_to(closest_city_tile.pos), 1)
                        action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                        actions.append(action)
                        unit_pos_next.append(next_pos)
                        
        else:
            unit_pos_next.append(unit.pos)
            
        if unit.pos in resource_tiles:
            resource_tiles.remove(unit.pos)
        if unit.pos in adjacent_resources:
            adjacent_resources.remove(unit.pos)
                                                      
    for k, city in player.cities.items():
        for ct0 in city.citytiles:
            if ct0.can_act():
                if ct0.pos not in cur_unit_pos and citytiles_sum > len(player.units):
                    action = ct0.build_worker()
                    actions.append(action)
                else:
                    action = ct0.research()
                    actions.append(action)
    
    return actions

In [ ]:
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run([agent, "simple_agent"])
env.render(mode="ipython", width=1200, height=800)

In [ ]:
%%writefile agent.py
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

### Define helper functions

# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles = find_resources(game_state)
    
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():
            # we want to mine only if there is space left in the worker's cargo
            if unit.get_cargo_space_left() > 0:
                # find the closest resource if it exists to this unit
                closest_resource_tile = find_closest_resources(unit.pos, player, resource_tiles)
                if closest_resource_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_resource_tile.pos))
                    actions.append(action)
            else:
                # find the closest citytile and move the unit towards it to drop resources to a citytile to fuel the city
                closest_city_tile = find_closest_city_tile(unit.pos, player)
                if closest_city_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                    actions.append(action)
    
    return actions

## Create a submission
Now we need to create a .tar.gz file with main.py (and agent.py) at the top level. We can then upload this!

In [ ]:
!tar -czf submission.tar.gz *

## Submit
Now open the /kaggle/working folder and find submission.tar.gz, download that file, navigate to the "MySubmissions" tab in https://www.kaggle.com/c/lux-ai-2021/ and upload your submission! It should play a validation match against itself and once it succeeds it will be automatically matched against other players' submissions. Newer submissions will be prioritized for games over older ones. Your team is limited in the number of succesful submissions per day so we highly recommend testing your bot locally before submitting.

## CLI Tool

There's a separate CLI tool that can also be used to run matches. It's recommended for Jupyter Notebook users to stick with just this notebook, and all other users including python users to follow the instructions on https://github.com/Lux-AI-Challenge/Lux-Design-2021

The other benefit however of using the CLI tool is that it generates much smaller, "stateless" replays and also lets you run a mini leaderboard on multiple bots ranked by various ranking algorithms

## Additional things to check out

Make sure you check out the Bot API at https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits

This documents what you can do using the starter kit files in addition to telling you how to use the annotation debug commands that let you annotate directly on a replay (draw lines, circle things etc.)

You can also run the following below to save a episode to a JSON replay file. These are the same as what is shown on the leaderbaord and you can upload the replay files to the online replay viewer https://2021vis.lux-ai.org/


For a local (faster) version of the replay viewer, follow installation instructions here https://github.com/Lux-AI-Challenge/Lux-Viewer-2021

In [ ]:
import json
replay = env.toJSON()
with open("replay.json", "w") as f:
    json.dump(replay, f)

## Suggestions / Strategies

There are a lot of places that could be improved with the agent we have in this tutorial notebook. Here are some!

- Using the build city action to build new cities and thus build new units
- Having cities perform research each turn to unlock new resources
- Writing collision-free code that lets units move smoothly around and through each other when navigating to targets
- Mining resources near your opponent's citytiles so they have less easy access to resources
- Using carts to deliver resources from far away clusters of wood, coal, uranium to a city in need
- Sending worker units over to the opponent's roads and pillaging them to slow down their agent
- Optimizing over how much to mine out of forests before letting them regrow so you can build more cities and get sustainable fuel